# Experimentation

Loading the datasets and cleaning. The following datasets are expected:
* `locations_clean_user_location.tsv`: The original provided list of raw locations with corresponding number of occurances
* In `/data`:
  * `cities1000.tsv`, cities with > 1000 pop. (GeoNames):
    * https://download.geonames.org/export/dump/cities1000.zip
    * Unzipped and renamed to `.tsv`
  * `countryInfo.tsv`, countries (GeoNames):
    * https://download.geonames.org/export/dump/countryInfo.txt
    * Unzipped and renamed to `.tsv`
  * `admin1CodesASCII.txt`, states and provinces (admin1) (GeoNames)
    * https://download.geonames.org/export/dump/admin1CodesASCII.txt

In [12]:
import os
import pandas as pd
import geopandas as gpd

current_dir = os.getcwd()
data_dir = "data"

## Load datasets

In [13]:
# Tweets user locations list
# Loading using pandas' read_csv (tab-deleted) to set 'tweet_id' dtype to int

tweets_user_locations = os.path.join(current_dir, "locations_clean_user_location.tsv")
df = pd.read_csv(tweets_user_locations, sep='\t', dtype={'tweet_id': int})
df.head(3)

,tweet_user_location,tweet_id
0,None,4994911
1,United States,190257
2,India,97652


In [14]:
# GeoNames (Cities with > 1000 inabitants)
# https://download.geonames.org/export/dump/cities1000.zip
# Loading using geopandas for geometry (usefulness tbd)

cities = os.path.join(current_dir, data_dir, "cities1000.tsv")
cities_df = gpd.read_file(cities)
cities_df.columns = cities_df.columns.str.lower() # lowercase headers
cities_df.head(3)

,geonameid,name,asciiname,altnames,latitude,longitude,featclass,featcode,country,cc2,admin1,admin2,admin3,admin4,population,elevation,gtopo30,timezone,moddate,geometry
0,3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,P,PPL,AD,,02,,,,1052.0,NaN,1721,Europe/Andorra,2012-11-03,POINT (1.65362 42.57952)
1,3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,P,PPLA,AD,,06,,,,8022.0,NaN,921,Europe/Andorra,2013-11-23,POINT (1.49129 42.46372)
2,3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,P,PPL,AD,,03,,,,2363.0,2050.0,2106,Europe/Andorra,2008-06-09,POINT (1.73361 42.54277)


In [15]:
# GeoNames (Countries info)
# https://download.geonames.org/export/dump/countryInfo.txt
# Loading using pandas' read_csv (tab-deleted), ignore lines 1-48

countries = os.path.join(current_dir, data_dir, "countryInfo.tsv")
countries_df = pd.read_csv(countries, sep='\t', header=49)
countries_df.head(3)

,#ISO,ISO3,ISO-Numeric,fips,Country,Capital,Area(in sq km),Population,Continent,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,geonameid,neighbours,EquivalentFipsCode
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,.ae,AED,Dirham,971,NaN,NaN,"ar-AE,fa,en,hi,ur",290557,"SA,OM",NaN
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS,.af,AFN,Afghani,93,NaN,NaN,"fa-AF,ps,uz-AF,tk",1149361,"TM,CN,IR,TJ,PK,UZ",NaN


In [16]:
# GeoNames (states and provinces, admin1)
# https://download.geonames.org/export/dump/admin1CodesASCII.txt
# Loading using pandas' read_csv (tab-deleted),
# Column names from https://download.geonames.org/export/dump/readme.txt
# 'code' is '<country>.<admin1 for country>'

admin1 = os.path.join(current_dir, data_dir, "admin1CodesASCII.txt")
admin1_df = pd.read_csv(admin1, sep='\t', names=['code', 'name', 'name ascii', 'geonameid'])
admin1_df.head(3)

,code,name,name ascii,geonameid
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131


## Helper functions

In [17]:
# Displays the percentage of tweets that have a 'geonameid'
def print_geonameid_completeness(df):
    all_tweets = df['tweet_id'].sum()
    geonameid_tweets = df[df.geonameid.notnull()]['tweet_id'].sum()
    print(f'{geonameid_tweets/all_tweets*100:.3f}%')

 ## Clean

In [18]:
# Make a copy of 'tweet_user_location' so we leave the original intact
df['tweet_user_location_copy'] = df['tweet_user_location']

# Discard specific 'tweet_user_location' strings
tweet_user_location_discard = ['None', '\\N']
df = df[~df['tweet_user_location'].isin(tweet_user_location_discard)]

# Discard locations that don't exist more than 2 times
df = df[df['tweet_id'] > 2]

# Filter out emojis and other symbols
# * https://stackoverflow.com/a/49986645
# * https://www.ling.upenn.edu/courses/Spring_2003/ling538/UnicodeRanges.html (Unicode symbol ranges)
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emojis: emoticons
        u"\U0001F300-\U0001F5FF"  # emojis: symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # emojis: transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # emojis: flags (iOS)
        u"\U00002700-\U000027BF"  # 'Dingbats' http://www.unicode.org/charts/PDF/U2700.pdf
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: deEmojify(x))

# Truncate leading and trailing spaces
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: x.strip())

# Truncate trailing "," and "." characters
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: x.rstrip(','))
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: x.rstrip('.'))

df

,tweet_user_location,tweet_id,tweet_user_location_copy
1,United States,190257,United States
2,India,97652,India
3,"London, England",77542,"London, England"
4,USA,67336,USA
5,London,66315,London
...,...,...,...
338210,N 52°27' 0'' / W 1°49' 0'',3,N 52°27' 0'' / W 1°49' 0''
338211,Villerupt-Luxembourg-Oslo-Stoc,3,Villerupt-Luxembourg-Oslo-Stoc
338212,Chicago ✈,3,Chicago
338213,Catch Me If You Can,3,Catch Me If You Can


## Countries

In [19]:
# Some locations are verbatim the name of a country, e.g.:
df[df['tweet_user_location_copy'] == 'Canada']

,tweet_user_location,tweet_id,tweet_user_location_copy
18,Canada,40858,Canada
1497,Canada,723,Canada
2846,Canada 🇨🇦,352,Canada
4756,Canada.,199,Canada
5333,Canada,177,Canada
9208,🇨🇦 Canada,100,Canada
14493,Canada 🇨🇦,63,Canada
17363,Canada🇨🇦,52,Canada
52402,🇨🇦Canada🇨🇦,17,Canada
78821,🍁 Canada 🍁,12,Canada


In [20]:
# df[df['tweet_user_location'].isin(simple_countries_df['Country'])]
# simple_countries_df = countries_df[['#ISO','Country', 'geonameid']]#.set_index('Country')

# Merge in country info (with goenameid) when there's an exact country match

# Keep the columns of countries_df we need.
simple_countries_df = countries_df[['#ISO','Country', 'geonameid']]
#df = pd.merge(df, simple_countries_df, how='left', left_on='tweet_user_location_copy', right_on='Country')
df

,tweet_user_location,tweet_id,tweet_user_location_copy
1,United States,190257,United States
2,India,97652,India
3,"London, England",77542,"London, England"
4,USA,67336,USA
5,London,66315,London
...,...,...,...
338210,N 52°27' 0'' / W 1°49' 0'',3,N 52°27' 0'' / W 1°49' 0''
338211,Villerupt-Luxembourg-Oslo-Stoc,3,Villerupt-Luxembourg-Oslo-Stoc
338212,Chicago ✈,3,Chicago
338213,Catch Me If You Can,3,Catch Me If You Can


In [21]:
#print_geonameid_completeness(df)

## Cities

In [22]:
#simple_cities_df = cities_df[['geonameid', 'name', 'asciiname', 'altnames']]
# df = pd.merge(df, simple_cities_df, how='left', left_on='tweet_user_location', right_on='name')
#df_copy = df[df['geonameid'].isnull()]
#pd.merge(df_copy, simple_cities_df, how='left', left_on='tweet_user_location_copy', right_on='name')

# NB: this can't work b/c cities name (unlike countries) aren't unique, e.g. there's a lot of "London"

In [23]:
df[df['tweet_user_location'].str.contains(',.+,')]

,tweet_user_location,tweet_id,tweet_user_location_copy
55,"Bogotá, D.C., Colombia",19367,"Bogotá, D.C., Colombia"
572,"Toronto, Ontario, Canada",2052,"Toronto, Ontario, Canada"
843,"Melbourne, Victoria, Australia",1398,"Melbourne, Victoria, Australia"
975,"Bogotá, DC, Colombia",1176,"Bogotá, DC, Colombia"
1111,"Ottawa, Ontario, Canada",1022,"Ottawa, Ontario, Canada"
...,...,...,...
338095,"Chicago, DC, Philly, NYC",3,"Chicago, DC, Philly, NYC"
338103,"Winterfel, Banlieue, Velodrome",3,"Winterfel, Banlieue, Velodrome"
338127,"Chapel Hill, NC/Syracuse, NY",3,"Chapel Hill, NC/Syracuse, NY"
338191,"Chicago, IL & Wangulei,S Sudan",3,"Chicago, IL & Wangulei,S Sudan"


In [24]:
df[df['tweet_user_location_copy'].str.count(',') > 2]

,tweet_user_location,tweet_id,tweet_user_location_copy
2988,"LA,Houston, NYC, ATL,Tampa",333,"LA,Houston, NYC, ATL,Tampa"
4053,"Gedung Graha Pena Lt.10, Jl. Raya Kebayoran La...",238,"Gedung Graha Pena Lt.10, Jl. Raya Kebayoran La..."
4698,"Paris, France/La Habana, Cuba/México/Roma, Italia",202,"Paris, France/La Habana, Cuba/México/Roma, Italia"
4753,"Lucerne, Suisse, Kinshasa,Bruxelles,Boston RDC",199,"Lucerne, Suisse, Kinshasa,Bruxelles,Boston RDC"
5556,"Durham, England, UK, EU till I die.",169,"Durham, England, UK, EU till I die"
...,...,...,...
336979,"Central, Southside, SW, VA",3,"Central, Southside, SW, VA"
337241,"EX:Sugar,A2B,eG,RL,CZ,aV,Eyy",3,"EX:Sugar,A2B,eG,RL,CZ,aV,Eyy"
337521,"Chicago, Austin, NYC, LA",3,"Chicago, Austin, NYC, LA"
337587,"Chicago, DC, Los Angeles, New York",3,"Chicago, DC, Los Angeles, New York"


In [25]:
test = "Toronto, Ontario, Canada, World"
test = "Toronto, Canada"
test.split(',')

['Toronto', ' Canada']

In [26]:
import numpy as np

def split_to_cols(num_parts, location):
    parts = location.split(',')
    if num_parts > len(parts):
        for i in range(num_parts - len(parts)):
            parts.insert(0, np.nan)
    else:
        for i in range(len(parts) - num_parts):
            parts.pop(0)
    return parts

def split_col_names(num_parts):
    return [f'element{i}' for i in range(num_parts)]

def new_series(num_parts, location):
    parts = split_to_cols(num_parts, location)
    return {f'el-{k}':parts[k] for k in range(num_parts)}

#col_parts = {f'el-{k}':parts[k] for k in range(num_parts)}
#col_parts

num_parts = 3  # num_parts-1 commas
split_to_cols(num_parts, test)
#split_col_names(num_parts)

# df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: x.rstrip(','))

[nan, 'Toronto', ' Canada']

In [39]:
# df[['column_new_1', 'column_new_2', 'column_new_3']] = pd.DataFrame([[np.nan, 'dogs', 3]], index=df.index)
test_df = df[df['tweet_user_location_copy'].str.count(',') > 2].head(10)
test_df

,tweet_user_location,tweet_id,tweet_user_location_copy
2988,"LA,Houston, NYC, ATL,Tampa",333,"LA,Houston, NYC, ATL,Tampa"
4053,"Gedung Graha Pena Lt.10, Jl. Raya Kebayoran La...",238,"Gedung Graha Pena Lt.10, Jl. Raya Kebayoran La..."
4698,"Paris, France/La Habana, Cuba/México/Roma, Italia",202,"Paris, France/La Habana, Cuba/México/Roma, Italia"
4753,"Lucerne, Suisse, Kinshasa,Bruxelles,Boston RDC",199,"Lucerne, Suisse, Kinshasa,Bruxelles,Boston RDC"
5556,"Durham, England, UK, EU till I die.",169,"Durham, England, UK, EU till I die"
5732,"Roma, Lazio, Italia, Europa",164,"Roma, Lazio, Italia, Europa"
6055,"Born: Roma, Italia, Cuban Exile Parents, Grew ...",154,"Born: Roma, Italia, Cuban Exile Parents, Grew ..."
6713,"Ca, Co, NY, Fl, Ak, USA",138,"Ca, Co, NY, Fl, Ak, USA"
6944,"Amherst, Pelham, Granby, MA",133,"Amherst, Pelham, Granby, MA"
7128,"Bhilai, Pune, Miraj, Riyadh",130,"Bhilai, Pune, Miraj, Riyadh"


In [40]:
import numpy as np

num_parts = 5
# for i in split_col_names(num_parts):
#     test_df[i] = np.nan
    
#test_df[split_col_names(num_parts)] = split_to_cols(num_parts, test)
#test_df[split_col_names(num_parts)] = test_df[split_col_names(num_parts)].apply(lambda x: [1,2,3])
#test_df

# test_df.apply(lambda x: x['tweet_user_location_copy'], axis=0)

# split_to_cols(num_parts, "Bogotá, D.C., Colombia")

# https://stackoverflow.com/a/16242202
test_df.tweet_user_location_copy.apply(lambda s: pd.Series(new_series(num_parts, s)))
pd.concat([test_df, test_df.tweet_user_location_copy.apply(lambda s: pd.Series(new_series(num_parts, s)))], axis=1)

,tweet_user_location,tweet_id,tweet_user_location_copy,el-0,el-1,el-2,el-3,el-4
2988,"LA,Houston, NYC, ATL,Tampa",333,"LA,Houston, NYC, ATL,Tampa",LA,Houston,NYC,ATL,Tampa
4053,"Gedung Graha Pena Lt.10, Jl. Raya Kebayoran La...",238,"Gedung Graha Pena Lt.10, Jl. Raya Kebayoran La...",Gedung Graha Pena Lt.10,Jl. Raya Kebayoran Lama No. 12,Kebayoran Lama,Jakarta Selatan,Indonesia 12210
4698,"Paris, France/La Habana, Cuba/México/Roma, Italia",202,"Paris, France/La Habana, Cuba/México/Roma, Italia",NaN,Paris,France/La Habana,Cuba/México/Roma,Italia
4753,"Lucerne, Suisse, Kinshasa,Bruxelles,Boston RDC",199,"Lucerne, Suisse, Kinshasa,Bruxelles,Boston RDC",Lucerne,Suisse,Kinshasa,Bruxelles,Boston RDC
5556,"Durham, England, UK, EU till I die.",169,"Durham, England, UK, EU till I die",NaN,Durham,England,UK,EU till I die
5732,"Roma, Lazio, Italia, Europa",164,"Roma, Lazio, Italia, Europa",NaN,Roma,Lazio,Italia,Europa
6055,"Born: Roma, Italia, Cuban Exile Parents, Grew ...",154,"Born: Roma, Italia, Cuban Exile Parents, Grew ...",Born: Roma,Italia,Cuban Exile Parents,Grew Up: NYC Boy,Now Loving Boston!
6713,"Ca, Co, NY, Fl, Ak, USA",138,"Ca, Co, NY, Fl, Ak, USA",Co,NY,Fl,Ak,USA
6944,"Amherst, Pelham, Granby, MA",133,"Amherst, Pelham, Granby, MA",NaN,Amherst,Pelham,Granby,MA
7128,"Bhilai, Pune, Miraj, Riyadh",130,"Bhilai, Pune, Miraj, Riyadh",NaN,Bhilai,Pune,Miraj,Riyadh
